In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
# 状態管理用のデータ構造
from pydantic import BaseModel,Field
from typing import Any


In [2]:
class AgentState(BaseModel):
    user_input: str = Field(...,description="user_input")
    response: str = Field(...,description="ai response")

In [3]:
# LLM のセットアップ
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

In [4]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
def call_llm(state: AgentState) -> dict[str,Any]:
    """LLM にユーザー入力を渡し、応答を取得"""
    prompt = ChatPromptTemplate.from_messages([
        ("human","{user_input}")
    ])
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"user_input": state.user_input})
    return {"response": response}

In [5]:
# グラフの構築
workflow = StateGraph(AgentState)

# ノードの追加
workflow.add_node("llm", call_llm)

# フローの設定
workflow.set_entry_point("llm")
workflow.add_edge("llm", END)

# ワークフローの作成
app = workflow.compile()

In [6]:
state = AgentState(user_input="あなたは優秀なエージェントです。カレーのレシピを考えて下さい。",response="")
result = app.invoke(state)

In [7]:
print("エージェントの応答:", result["response"])

エージェントの応答: もちろん、美味しいカレーのレシピを考えてみましょう。以下は、基本的なチキンカレーのレシピです。お好みで具材やスパイスをアレンジしてみてください。

### チキンカレーのレシピ

#### 材料（4人分）
- 鶏もも肉：500g（一口大に切る）
- 玉ねぎ：2個（薄切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- トマト缶：1缶（400g）
- ヨーグルト：100g
- カレー粉：大さじ2
- クミン：小さじ1
- コリアンダー：小さじ1
- ターメリック：小さじ1
- ガラムマサラ：小さじ1
- 塩：適量
- 胡椒：適量
- サラダ油：大さじ2
- 水：200ml
- パクチーや香菜（お好みで、飾り用）

#### 作り方
1. **準備**：
   - 鶏もも肉に塩と胡椒をふりかけ、ヨーグルトを加えて混ぜ合わせ、30分ほどマリネしておきます。

2. **炒める**：
   - 鍋にサラダ油を熱し、クミンを加えて香りが立つまで炒めます。
   - 次に、玉ねぎを加えて、しんなりするまで中火で炒めます。にんにくと生姜を加えてさらに1分ほど炒めます。

3. **スパイスを加える**：
   - カレー粉、コリアンダー、ターメリックを加えて、混ぜ合わせながら1〜2分炒め、香りを引き出します。

4. **トマトと鶏肉を加える**：
   - トマト缶を加え、軽く煮詰めます。
   - マリネした鶏肉を加え、全体をよく混ぜ合わせます。

5. **煮込む**：
   - 水を加え、沸騰したら弱火にし、蓋をして20〜30分ほど煮込みます。時々かき混ぜながら、鶏肉が柔らかくなるまで煮込みます。

6. **仕上げ**：
   - ガラムマサラを加えて全体を混ぜ、塩で味を調整します。

7. **盛り付け**：
   - お皿に盛り付け、お好みでパクチーや香菜を散らして完成です。

このレシピは基本的なものなので、好きな具材やスパイスを加えて、オリジナルのカレーを楽しんでください。お好みでジャガイモやナス、ほうれん草などを加えても美味しいです。
